# 2019/12/03 CoE 202 Activity 5

### **Name Classification**<br/>

**Professor: Yong Hoon, Lee**</br>

**TA : Seungjun moon, Beomgu Kang**

In [1]:
import tensorflow as tf
import numpy as np
import os.path
import string

model_save_path = 'tmp/model.ckpt'
tf.reset_default_graph()

### Hyperparameters

In [0]:
learning_rate       = 0.005
n_epoch             = 200
n_hidden            = 128 # hidden layer features
max_sequence_length = 19 # maximum number of characters is 19

In [0]:
all_letters = string.ascii_letters + " .,;'"
n_input     = len(all_letters)
alphabet    = all_letters
ethnicities = ['Chinese', 'Japanese', 'Vietnamese', 'Korean', 'Arabic','Czech','Dutch','English','French','German','Greek','Irish','Italian','Polish','Portuguese','Russian','Scottish','Spanish']
n_classes   = len(ethnicities) # the number of classes

name_strings, ethnicity_strings, str_list, names_list, ethnicity_list = [], [], [], [], []

## Define functions

In [0]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [0]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [0]:
def name_one_hot(name, max_sequence_length):
    result = []
    for char in name:
        v = np.zeros(n_input, dtype=np.int) # count space as a character
        v[alphabet.index(char)] = 1
        result.append(v)
    while len(result) < max_sequence_length:
        result.append(np.zeros(n_input, dtype=np.int))
    result = np.array(result)
    return result

In [0]:
def ethnicity_one_hot(ethnicity):
    v = np.zeros(n_classes, dtype=np.int)
    v[ethnicities.index(ethnicity)] = 1
    return v

## Data load 

In [0]:
with open('names_revised.csv', 'r') as csv:
    for line in csv:       
        l = [s.strip() for s in line.split(',')] # lowercase L, not capital i , l['name', 'ehnicity']
        if(l[1] in ethnicities):
            name_strings.append(l[0])
            ethnicity_strings.append(l[1])
            if len(l[0]) > max_sequence_length:
                l[0] = l[0][:max_sequence_length]
            names_list.append(name_one_hot(l[0], max_sequence_length)) # one-hot vector of each characters of name
            ethnicity_list.append(ethnicity_one_hot(l[1])) # one-hot vector of ethnicity

## Training - Test Seperation

In [0]:
rng_state = np.random.get_state() # use the same random number generator state
np.random.shuffle(names_list)     # when shuffling the two lists
np.random.set_state(rng_state)    # they are effectively shuffled in parallel so that inputs still correspond to outputs after shuffling
np.random.shuffle(ethnicity_list)

In [0]:
size = len(names_list) 
train_size = np.int(size*2/3) 

training_X = np.array(names_list[:train_size])
training_y = np.array(ethnicity_list[:train_size])
testing_X = np.array(names_list[train_size:])
testing_y = np.array(ethnicity_list[train_size:])

## Build a model

In [0]:
X = tf.placeholder(tf.float32, [None, max_sequence_length, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [12]:
x = tf.reshape(X, [-1, max_sequence_length * n_input])
a= max_sequence_length * n_input
print (x)
w_init = tf.variance_scaling_initializer()
b_init = tf.constant_initializer(0.)
#print (n_classes)
## 1st hidden layer
w1 = weight_variable([a, 256])                               # weight for 1st hidden layer which have 256 units
b1 = bias_variable ([256])                             # bias for 1st hidden layer which have 256 units
#h  = tf.keras.layers.dot()
#print(w1) 
h= tf.matmul (x, w1) + b1;                              # matrix multiplication
h  = tf.nn.relu (h)                               # relu activation


print (h)

## 2nd hidden layer
w2 = weight_variable([256, 256]) 
print (w2)                             # weight for 2nd hidden layer which have 256 units
b2 =  bias_variable ([256])                               # bias for 2nd hidden layer which have 256 units
h  = tf.matmul (h, w2) +b2;                               # matrix multiplication
h  = tf.nn.relu (h)                               # relu activation
print (h)
#print (w3)
## output layer
w3 = weight_variable([256, n_classes])                               # weight for output layer which have 256 units

y_ = tf.matmul(h, w3)
print (y_)

Tensor("Reshape:0", shape=(?, 1083), dtype=float32)
Tensor("Relu:0", shape=(?, 256), dtype=float32)
<tf.Variable 'Variable_2:0' shape=(256, 256) dtype=float32_ref>
Tensor("Relu_1:0", shape=(?, 256), dtype=float32)
Tensor("MatMul_2:0", shape=(?, 18), dtype=float32)


In [43]:
out_weights = weight_variable([n_hidden, n_classes])
out_biases = bias_variable([n_classes])
# Basic RNN
#cells = tf.contrib.rnn.BasicRNNCell(num_units = 128)
# LSTM
cells = tf.contrib.rnn.BasicLSTMCell(num_units = 128)
# GRU
#cells = tf.contrib.rnn.GRUCell(num_units = 128)

outputs, states = tf.nn.dynamic_rnn(cells, X, dtype=tf.float32)

y_ = tf.matmul(outputs[:,-1,:], out_weights) + out_biases # predict y based on final rnn output
print (y_)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Tensor("add_23:0", shape=(?, 18), dtype=float32)


In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_, labels=y))
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [0]:
# Evaluation
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
# Softmax
pred = tf.nn.softmax(y_)

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

## Train a model

In [0]:
sess = tf.InteractiveSession()
sess.run(init)

In [18]:


for _ in range(n_epoch+1):
    if _%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={X:training_X, y:training_y})
        sess.run(train_step, feed_dict={X: training_X, y: training_y})
        print("step %d, training accuracy %g"%(_, train_accuracy))
        test_accuracy = accuracy.eval(feed_dict={X:testing_X, y:testing_y})
        print("testing accuracy", test_accuracy)
        if _== 200:
          print ("=====================================")
          test_accuracy = accuracy.eval(feed_dict={X:testing_X, y:testing_y})
          print("testing accuracy", test_accuracy)
saver.save(sess, model_save_path)
print("Model saved in file: %s" % model_save_path)

step 0, training accuracy 0.210613
testing accuracy 0.46779257
step 10, training accuracy 0.468984
testing accuracy 0.47003436
step 20, training accuracy 0.470703
testing accuracy 0.5278733
step 30, training accuracy 0.53565
testing accuracy 0.5096398
step 40, training accuracy 0.515845
testing accuracy 0.4955911
step 50, training accuracy 0.502018
testing accuracy 0.50141984
step 60, training accuracy 0.509417
testing accuracy 0.5230907
step 70, training accuracy 0.53296
testing accuracy 0.54954416
step 80, training accuracy 0.563154
testing accuracy 0.5607532
step 90, training accuracy 0.579073
testing accuracy 0.57823944
step 100, training accuracy 0.597085
testing accuracy 0.60424453
step 110, training accuracy 0.620628
testing accuracy 0.6248692
step 120, training accuracy 0.64432
testing accuracy 0.64190704
step 130, training accuracy 0.66009
testing accuracy 0.6441489
step 140, training accuracy 0.663901
testing accuracy 0.65236884
step 150, training accuracy 0.672496
testing ac

In [25]:
i=0
while i<8:
    input_name = input('Enter a last name (max 19 letters):')
   
    while len(input_name) > max_sequence_length or len(input_name) == 0:
        input_name = raw_input('Invalid input. Enter a last name (max 19 letters):')
   
    result=pred.eval(feed_dict={X: np.expand_dims(name_one_hot(input_name, 19), axis=0)})[0]
    idx = np.argsort(result)[::-1]
    print("\n(%s): %.4f" % (ethnicities[idx[0]], result[idx[0]]))
    print("(%s): %.4f" % (ethnicities[idx[1]], result[idx[1]]))
    print("(%s): %.4f" % (ethnicities[idx[2]], result[idx[2]]))
    print("==========================================")
    i=i+1

Enter a last name (max 19 letters):Pyatykh

(Russian): 0.9922
(Czech): 0.0051
(English): 0.0018
Enter a last name (max 19 letters):Khachaturov

(Russian): 0.9901
(Czech): 0.0062
(Polish): 0.0020
Enter a last name (max 19 letters):Cherkashin

(Russian): 0.9245
(Czech): 0.0484
(Polish): 0.0120
Enter a last name (max 19 letters):Kuzmin

(Czech): 0.3026
(German): 0.2296
(Dutch): 0.2143
Enter a last name (max 19 letters):Bautin

(Russian): 0.7453
(English): 0.1620
(Irish): 0.0278
Enter a last name (max 19 letters):Samoylenko

(Russian): 0.9370
(Czech): 0.0444
(Polish): 0.0101
Enter a last name (max 19 letters):Ovsienko

(Russian): 0.4247
(Czech): 0.3134
(Polish): 0.1372
Enter a last name (max 19 letters):Mishukov

(Russian): 0.9926
(Czech): 0.0053
(Polish): 0.0010


## 4. Report

### a. Use GRU, LSTM and Simple RNN functions for training . Compare each of results.

### b. Replace the RNN with DNN as below.

In [14]:
x = tf.reshape(X, [-1, max_sequence_length * n_input])
a= max_sequence_length * n_input

w_init = tf.variance_scaling_initializer()
b_init = tf.constant_initializer(0.)


w1 = weight_variable([a, 256])                               # weight for 1st hidden layer which have 256 units
b1 = bias_variable ([256])                             # bias for 1st hidden layer which have 256 units 
h= tf.matmul (x, w1) + b1;                              # matrix multiplication
h  = tf.nn.relu (h)                               # relu activation


print (h)

## 2nd hidden layer
w2 = weight_variable([256, 256])                            # weight for 2nd hidden layer which have 256 units
b2 =  bias_variable ([256])                               # bias for 2nd hidden layer which have 256 units
h  = tf.matmul (h, w2) +b2;                               # matrix multiplication
h  = tf.nn.relu (h)                               # relu activation

## output layer
w3 = weight_variable([256, n_classes])                               # weight for output layer which have 256 units

y_ = tf.matmul(h, w3)


Tensor("Reshape_1:0", shape=(?, 1083), dtype=float32)
Tensor("Relu_2:0", shape=(?, 256), dtype=float32)
<tf.Variable 'Variable_6:0' shape=(256, 256) dtype=float32_ref>
Tensor("Relu_3:0", shape=(?, 256), dtype=float32)
Tensor("MatMul_4:0", shape=(?, 18), dtype=float32)


### Submission (Due: Dec. 10 Tue.)
Submit your report by Tuesday, December 10 to **"june1212@kaist.ac.kr"**